In [ ]:
import os
import pandas as pd

root_dir = os.getcwd()
df_benchmark = pd.read_csv(os.path.join(root_dir, "out", "benchmark.csv"))

df_benchmark = df_benchmark.rename(columns={
    'graph_name': 'Grafo',
    'vertices': 'Vértices',
    'edges': 'Arestas',
    'ff_sol': 'FF K',
    'ff_time': 'FF Tempo (ms)',
    'wp_sol': 'WP K',
    'wp_time': 'WP Tempo (ms)',
    'ldo_sol': 'LDO K',
    'ldo_time': 'LDO Tempo (ms)',
    'ido_sol': 'IDO K',
    'ido_time': 'IDO Tempo (ms)',
    'dsatur_sol': 'DSATUR K',
    'dsatur_time': 'DSATUR Tempo (ms)',
    'rlf_sol': 'RLF K',
    'rlf_time': 'RLF Tempo (ms)',
})

df_benchmark_avg = df_benchmark.groupby(['Grafo', 'Vértices', 'Arestas'], as_index=False)\
  .mean()\
  .astype({
    'FF K': 'int',
    'WP K': 'int',
    'LDO K': 'int',
    'IDO K': 'int',
    'DSATUR K': 'int',
    'RLF K': 'int'
  })

df_benchmark_avg.to_csv(os.path.join(root_dir, "out", "benchmark_avg.csv"))

display(df_benchmark)
display(df_benchmark_avg)

In [ ]:
with open(os.path.join(root_dir, "manuscript", "results_tab.tex"), 'w') as f:
    f.write(df_benchmark_avg.style.format(precision=2).hide(axis="index").to_latex())

In [ ]:
# C2000.5 del
# C4000.5 del
# dsjc1000.1 20
# dsjc1000.5 83
# dsjc1000.9 222
# dsjc250.5 28
# dsjc500.1 12
# dsjc500.5 48
# dsjc500.9 126
# dsjr500.1c 84
# dsjr500.5 122
# flat1000_50_0	 50
# flat1000_60_0 60
# flat1000_76_0 76
# flat300_28_0	28
# latin_square 97
# le450_25c 25
# le450_25d 25
# r1000.1c 98
# r1000.5 234
# r250.5 65

In [ ]:
df_Graph_K = df_benchmark.groupby(['Grafo'], as_index=False)[['FF K', 'WP K', 'LDO K', 'IDO K', 'DSATUR K', 'RLF K']].first()

df_best_K = pd.DataFrame({
  'Grafo': ['C2000.5', 'C4000.5', 'dsjc1000.1', 'dsjc1000.5', 'dsjc1000.9', 'dsjc250.5', 'dsjc500.1', 'dsjc500.5', 'dsjc500.9', 'dsjr500.1c', 'dsjr500.5', 'flat1000_50_0', 'flat1000_60_0', 'flat1000_76_0', 'flat300_28_0', 'latin_square', 'le450_25c', 'le450_25d', 'r1000.1c', 'r1000.5', 'r250.5'],
  'Ideal': [146, 260, 20, 83, 222, 28, 12, 48, 126, 84, 122, 50, 60, 76, 28, 97, 25, 25, 98, 234, 65]
})

df_Graph_K = df_Graph_K.merge(df_best_K, how='right').dropna()

for column in [c for c in df_Graph_K.columns if c.endswith(' K')]:
  df_Graph_K[column] = df_Graph_K['Ideal'] / df_Graph_K[column]

df_Graph_K


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

fig, ax = plt.subplots(figsize=(8, 5))
df_Graph_K[[c for c in df_Graph_K.columns if c.endswith(' K')]].plot.box(
    ax=ax,
    showmeans=True,       # draw the mean
    meanline=True,        # as a line (not a marker)
    meanprops={           # style for the mean‐lines
        "linestyle": "--",
        "color": "black",
        "linewidth": 1.2,
    },
    boxprops={"linewidth": 1},   # optional styling
    whiskerprops={"linewidth": 1},
    capprops={"linewidth": 1},
    medianprops={"color": "cornflowerblue", "linewidth": 1.5}
)

ax.yaxis.set_major_formatter(PercentFormatter(xmax=1.0, decimals=0))

ax.set_ylabel(r"Acurácia (\%)")
ax.set_title("Comparação da Acurácia dos Algoritmos")
# plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

sns.set_theme(style="ticks")

f, ax = plt.subplots(figsize=(8, 6))

df_melt = df_Graph_K.melt(
    id_vars="Grafo",
    value_vars=["FF K","WP K","LDO K","IDO K","DSATUR K","RLF K"],
    var_name="Método",
    value_name="Acurácia"
)

# horizontal
sns.boxplot(
    data=df_melt,
    x="Acurácia",
    y="Método",
    # whis=[0, 100],     # show min/max
    width=0.6,
    palette="vlag",
    hue="Método",
    legend=False,
    ax=ax,
    showmeans=True
)

# overplot
sns.stripplot(
    data=df_melt,
    x="Acurácia",
    y="Método",
    size=4,
    color=".3",
    jitter=True,
    ax=ax
)

# x‐axis percentual
ax.xaxis.set_major_formatter(
    mtick.PercentFormatter(xmax=1.0, decimals=0)
)

# x-limits
ax.set_xlim(min(df_melt['Acurácia']) - 0.025, max(df_melt['Acurácia']) + 0.025)


font = {'fontsize': 15}

ax.set_xlabel(r"Acurácia (%)", **font)
ax.set_ylabel("", **font) 
ax.xaxis.grid(True)

sns.despine(left=True, trim=False)

plt.tight_layout()
plt.savefig(os.path.join(root_dir, "manuscript", "img", "boxplot.png"), transparent=True, bbox_inches='tight', dpi=400)
plt.show()